In [149]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, balanced_accuracy_score
from sklearn.linear_model import LogisticRegression
import pickle


weather = pd.read_csv('./dataset/clean-dataset.csv')
rows, columns = weather.shape
print(f"Rows: {rows}, Columns: {columns}")

Rows: 142146, Columns: 25


### Conversion de variables

In [131]:
weather.head(5)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,DailyTempRange,YearMonth
0,2008-07-02,Adelaide,12.7,15.8,0.8,1.4,7.8,SW,35.0,SSW,...,1022.4,1022.6,NaN,NaN,13.7,15.5,0.0,0.0,3.1,2008-07
1,2008-07-03,Adelaide,6.2,15.1,0.0,1.8,2.1,W,20.0,NNE,...,1027.8,1026.5,NaN,NaN,9.3,13.9,0.0,0.0,8.9,2008-07
2,2008-07-04,Adelaide,5.3,15.9,0.0,1.4,8.0,NNE,30.0,NNE,...,1028.7,1025.6,NaN,NaN,10.2,15.3,0.0,0.0,10.6,2008-07
3,2008-07-06,Adelaide,11.3,15.7,0.0,1.4,1.5,NNW,52.0,NNE,...,1019.5,1016.2,NaN,NaN,13.0,14.4,0.0,1.0,4.4,2008-07
4,2008-07-07,Adelaide,7.6,11.2,16.2,4.6,1.1,WSW,46.0,WNW,...,1015.9,1017.9,NaN,NaN,9.8,9.3,1.0,1.0,3.6,2008-07


In [132]:
weather.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142146 entries, 0 to 142145
Data columns (total 25 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Date            142146 non-null  object 
 1   Location        142146 non-null  object 
 2   MinTemp         142146 non-null  float64
 3   MaxTemp         142146 non-null  float64
 4   Rainfall        142146 non-null  float64
 5   Evaporation     142146 non-null  float64
 6   Sunshine        142146 non-null  float64
 7   WindGustDir     132820 non-null  object 
 8   WindGustSpeed   142146 non-null  float64
 9   WindDir9am      132135 non-null  object 
 10  WindDir3pm      138370 non-null  object 
 11  WindSpeed9am    142146 non-null  float64
 12  WindSpeed3pm    142146 non-null  float64
 13  Humidity9am     142146 non-null  float64
 14  Humidity3pm     142146 non-null  float64
 15  Pressure9am     142146 non-null  float64
 16  Pressure3pm     142146 non-null  float64
 17  Cloud9am  

#### Conversion de Date

In [133]:
# weather["Date"] = pd.to_datetime(weather["Date"])

#### Conversión de RainToday y RainTomorrow a Boolean

In [134]:
weather["RainToday"] = weather["RainToday"].apply(lambda x: True if x == 1.0 else False)
weather["RainTomorrow"] = weather["RainTomorrow"].apply(lambda x: True if x == 1.0 else False)


#### Conversion de variables categoricas

In [135]:
variables_categoricas = ["Location", "WindGustDir", "WindDir3pm", "WindDir9am"]
weather_with_dummies = pd.get_dummies(weather, columns=variables_categoricas, drop_first=True)

### Resultados post conversion de variables

In [136]:
weather_with_dummies.head(5)

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,WindDir9am_NNW,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW
0,2008-07-02,12.7,15.8,0.8,1.4,7.8,35.0,13.0,15.0,75.0,...,False,False,False,False,False,True,False,False,False,False
1,2008-07-03,6.2,15.1,0.0,1.8,2.1,20.0,2.0,11.0,81.0,...,False,False,False,False,False,False,False,False,False,False
2,2008-07-04,5.3,15.9,0.0,1.4,8.0,30.0,6.0,13.0,71.0,...,False,False,False,False,False,False,False,False,False,False
3,2008-07-06,11.3,15.7,0.0,1.4,1.5,52.0,15.0,22.0,62.0,...,False,False,False,False,False,False,False,False,False,False
4,2008-07-07,7.6,11.2,16.2,4.6,1.1,46.0,17.0,13.0,83.0,...,False,False,False,False,False,False,False,False,True,False


In [137]:
weather.shape

(142146, 25)

In [138]:
weather_with_dummies.shape

(142146, 114)

In [139]:
weather_with_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142146 entries, 0 to 142145
Columns: 114 entries, Date to WindDir9am_WSW
dtypes: bool(95), float64(17), object(2)
memory usage: 33.5+ MB


### Separacion de Dataset

In [140]:
weather_with_dummies.drop(columns="Date", inplace=True)
weather_with_dummies.drop(columns="YearMonth", inplace=True)

X = weather_with_dummies.drop(columns="RainTomorrow")
y = weather_with_dummies["RainTomorrow"]

X.fillna(X.mean(), inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Escalado de variables numericas

In [141]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Aplicar la misma transformación al dataset de prueba
X_test_scaled = scaler.transform(X_test)

### Modelado y Evaluacion

#### Determine dos métricas de evaluación que considere importante medir para este problema. Justifique su elección.

Para predecir si va a llover mañana, las métricas de evaluación más relevantes podrían ser las siguientes:

 1. *⁠Exactitud Balanceada*

    En este caso de uso es interesante evaluar los modelos según su sensibilidad (tasa de verdaderos positivos) y según su especificidad (tasa de verdaderos negativos). Sin embargo, estas metricas traen problemas en los casos donde existe un desbalance de clases. En el presente dataset existe tal desbalance ya que, para ciertas ciudades, son muchos mas los dias en los que no llueve que los dias donde si se da este fenómeno meteorológico. Dicho esto, elejimos calcular la "Exactitud Balanceada"
    
 2. *Recuperación*

    Seleccionamos "Recuperación" como segunda metrica relevante para medir los modelos ya que nos interesa minimizar la cantidad de Falsos Negativos. Es decir, preferimos que el modelo indique que Si va a llover aunque en realidad despues No llueva y no al revés.

### Implemente un modelo de base usando como predicción que determine que llueve mañana si hoy llueve, y si hoy no llueve mañana no va a llover.

In [146]:
#TODO: Chequera que este bien hacer esto con weather dummys

# Aplicar la lógica de predicción: Si llueve hoy, predice que lloverá mañana

y_pred_test = X_test['RainToday']

# Calcular el recall y la precisión balanceada para el conjunto de prueba
recall_base_model = recall_score(y_test, y_pred_test, average='binary')
balanced_accuracy_base_model = balanced_accuracy_score(y_test, y_pred_test)

print(f"Base Model - Recall: {recall_base_model}")
print(f"Base Model - Balanced Accuracy: {balanced_accuracy_base_model}")



Base Model - Recall: 0.46675783715548075
Base Model - Balanced Accuracy: 0.65580936097016


#### Implemente y entrene un modelo de k Nearest Neighbors (kNN) para predecir si va a llover mañana. Utilice la distancia euclidiana como medida de distancia entre vecinos. Ajuste el parámetro k utilizando técnicas como la validación cruzada (5 folds) para optimizar el rendimiento del modelo (utilice como función de optimización una de las métricas definidas en el punto anterior).

In [116]:

knn = KNeighborsClassifier(metric='euclidean')

# Definir los parámetros de la búsqueda en grilla
param_grid = {'n_neighbors': list(range(1, 9))}  # Probamos k entre 1 y 9


# Configurar la validación cruzada con GridSearchCV
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='balanced_accuracy')# After fitting GridSearchCV
grid_search.fit(X_train_scaled, y_train)

# Seleccionar el mejor modelo
best_model = grid_search.best_estimator_

# Hacer predicciones con el mejor modelo
predictions = best_model.predict(X_train_scaled)
grid_search.fit(X_train_scaled, y_train)

# Obtener los mejores parámetros
best_k = grid_search.best_params_['n_neighbors']

print(f"Mejor valor de k: {best_k}")



Mejor valor de k: 1


In [121]:
best_model = grid_search.best_estimator_

predictions_knn = best_model.predict(X_test_scaled)

recall_knn = recall_score(y_test, predictions_knn)
print(f"Recall (Recovery): {recall_knn}")

balanced_accuracy_knn = balanced_accuracy_score(y_test, predictions_knn)
print(f"Balanced Accuracy: {balanced_accuracy_knn}")


Recall (Recovery): 0.42278560908899643
Balanced Accuracy: 0.6459392466955032


### Implemente y entrene un modelo de regresión logística para predecir la misma variable objetivo.

In [119]:
model_logistic = LogisticRegression()
model_logistic.fit(X_train_scaled, y_train)

predictions_logistic = model_logistic.predict(X_test_scaled)

# Calcular el recall
recall_logistic = recall_score(y_test, predictions_logistic, average='binary')

# Calcular la precisión balanceada
balanced_accuracy_logistic = balanced_accuracy_score(y_test, predictions_logistic)

print(f"Recall: {recall_logistic}")
print(f"Balanced Accuracy: {balanced_accuracy_logistic}")


Recall: 0.5048390490216705
Balanced Accuracy: 0.7245964814786667


### Evalúe los dos modelos con las métricas definidas al principio. Discuta las fortalezas y debilidades de cada enfoque en el contexto específico de este problema comparándolos con el modelo de base.

### Evaluación de Modelos: KNN vs. Regresión Logística vs. Modelo Base

#### Resultados

- **Modelo Base:**
  - Recall: 0.4668
  - Balanced Accuracy: 0.6558

- **KNN:**
  - Recall: 0.4228
  - Balanced Accuracy: 0.6459

- **Regresión Logística:**
  - Recall: 0.5048
  - Balanced Accuracy: 0.7246

### Análisis y Comparación

#### **Recall:**

- **Modelo Base (0.4668):** El modelo base, que predice que va a llover mañana si hoy llueve, tiene un **recall** moderado. Captura aproximadamente el 46.68% de los días lluviosos correctamente. Aunque es un modelo simple, proporciona un punto de referencia razonable.

- **KNN (0.4228):** El modelo KNN tiene un recall inferior al del modelo base. Esto sugiere que KNN es menos efectivo para identificar los días lluviosos en comparación con el modelo base, lo que es una debilidad significativa para este tipo de problema, donde detectar lluvia es crucial.

- **Regresión Logística (0.5048):** La regresión logística tiene el mejor recall entre los tres modelos, superando tanto a KNN como al modelo base. Esto indica que la regresión logística es más efectiva para detectar días lluviosos, lo que es una ventaja en escenarios donde es crucial minimizar los falsos negativos.

#### **Balanced Accuracy:**

- **Modelo Base (0.6558):** El modelo base tiene una balanced accuracy decente, lo que significa que su rendimiento general, teniendo en cuenta ambas clases (lluvia y no lluvia), es bastante equilibrado, aunque no sobresaliente.

- **KNN (0.6459):** El rendimiento global de KNN, medido por la balanced accuracy, es ligeramente inferior al del modelo base. Esto sugiere que, en términos generales, KNN no supera la simplicidad del modelo base, lo que podría limitar su utilidad en este contexto.

- **Regresión Logística (0.7246):** La regresión logística muestra una balanced accuracy superior, lo que indica un rendimiento global significativamente mejor que el modelo base y KNN. Esto sugiere que la regresión logística no solo identifica mejor los días lluviosos, sino que también maneja bien ambas clases.

### Fortalezas y Debilidades

#### **Modelo Base:**
- **Fortalezas:**
  - **Simplicidad:** Es extremadamente simple y fácil de interpretar.
  - **Desempeño Razonable:** Tiene un desempeño razonable en términos de recall y balanced accuracy, lo que lo convierte en un buen punto de referencia.

- **Debilidades:**
  - **Limitación en Capturar Complejidad:** No tiene la capacidad de capturar patrones complejos en los datos, lo que limita su efectividad cuando las condiciones climáticas no siguen un patrón simple.

#### **KNN:**
- **Fortalezas:**
  - **Flexibilidad:** Es un modelo no paramétrico que puede captar relaciones no lineales en los datos.
  
- **Debilidades:**
  - **Desempeño Inferior:** Tiene un recall y una balanced accuracy inferiores al modelo base, lo que sugiere que, en este caso, KNN no aporta mejoras significativas.
  - **Sensibilidad a la Escala:** Puede ser sensible a la escala de las características, lo que podría requerir un preprocesamiento cuidadoso.

#### **Regresión Logística:**
- **Fortalezas:**
  - **Desempeño Superior:** Supera tanto a KNN como al modelo base en recall y balanced accuracy, lo que sugiere que maneja mejor la tarea de clasificación.
  - **Interpretable:** Es fácil de interpretar y proporciona probabilidades, lo que es útil para entender la incertidumbre en las predicciones.

- **Debilidades:**
  - **Asunción de Linealidad:** Asume una relación lineal entre las características y la variable objetivo, lo que puede no ser ideal en todos los escenarios.

### Conclusión

En comparación con el modelo base, **KNN** no logra mejorar el desempeño, lo que sugiere que, para este problema, KNN no es la mejor opción. **La regresión logística**, en cambio, supera tanto a KNN como al modelo base en términos de recall y balanced accuracy, lo que la convierte en la mejor opción entre los tres modelos para predecir si lloverá mañana. 

Esto sugiere que, aunque un modelo base simple puede ser un punto de partida sólido, la regresión logística ofrece mejoras significativas en la capacidad de predicción y equilibrio entre las clases, lo que es crucial para aplicaciones climáticas donde la precisión en la predicción de lluvia es esencial.

In [151]:
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

# Guardar en archivos CSV
train_data.to_csv('./dataset/train_data.csv', index=False)
test_data.to_csv('./dataset/test_data.csv', index=False)

with open('./models/model_logistic.pkl', 'wb') as file:
    pickle.dump(model_logistic, file)